In [1]:
from pyoptools.all import *

Loading component library Edmund  from files  ['/usr/lib/python3/dist-packages/pyoptools/raytrace/library/Edmund/SphOptics.cmp', '/usr/lib/python3/dist-packages/pyoptools/raytrace/library/Edmund/SphOptics1.cmp']
Loading component library Thorlabs  from files  ['/home/richi/.pyoptools/library/Thorlabs/SphOptics1.cmp']
Loading component library EdmundHome  from files  ['/home/richi/.pyoptools/library/EdmundHome/SphOptics.cmp']


In [59]:
def convert(d):
    try:
        return int(d)
    except ValueError:
        try:
            return float(d)
        except ValueError:
            return d

def zmx_read(fn):

    f=open(fn,"rU")
    data=f.read()
    return zmx_parse(data)

##Codigo tomado y modificado de https://github.com/jordens/rayopt/blob/master/rayopt/zemax.py
from struct import Struct
import numpy as np

def zmf2dict(fn):
    """Función que lee una librería de Zemax (archivo con terminación zmf), y genera un diccionario con las descripciones
    de cada componente. La llave es la referencia de cada componente
    """
    f=open(fn,"rb")
    rd={}
    head = Struct("<I")
    lens = Struct("<100sIIIIIIIdd")
    shapes = "?EBPM"
    version, = head.unpack(f.read(head.size))
    assert version in (1001, )
    while True:
        li = f.read(lens.size)
        if len(li) != lens.size:
            if len(li) > 0:
                print(f, "additional data", repr(li))
            break
        li = list(lens.unpack(li))
        li[0] = li[0].decode("latin1").strip("\0")
        li[3] = shapes[li[3]]
        description = f.read(li[7])
        assert len(description) == li[7]
        description = zmf_obfuscate(description, li[8], li[9])
        description = description.decode("latin1")
        assert description.startswith("VERS {:06d}\n".format(li[1]))
        rd[li[0]]=description
    return rd


def zmf_obfuscate(data, a, b):
    iv = np.cos(6*a + 3*b)
    iv = np.cos(655*(np.pi/180)*iv) + iv
    p = np.arange(len(data))
    k = 13.2*(iv + np.sin(17*(p + 3)))*(p + 1)
    k = (int(("{:.8e}".format(_))[4:7]) for _ in k)
    #data = np.fromstring(data, np.uint8)
    data = np.frombuffer(data, np.uint8).copy()
    data ^= np.fromiter(k, np.uint8, len(data))
    return data.tostring()

def find_key(key,lines):
    try:
        rv = list(filter(lambda x:x.startswith(key),lines))[0].split(" ",1)[1]
    except IndexError:
        # No se encontró la llave 
        rv=""
    return rv

def zmx2pyoptools(libdata,key):
    
    data=libdata[key]

    surfaces=data.splitlines()
    #Separar encabezado de los datos
    header=[]
    # Interpretar el encabezado
    while True:
        if surfaces[0].startswith("SURF"):
            break
        line=surfaces.pop(0)
        header.append(line)
    
    pyot={}
    lens_data={}

    description=find_key("NAME",header)
    lens_data["description"]=description
    
    #Aun no se que hacer con el mode
    mode=find_key("MODE",header)
    
    if mode !="SEQ":
        raise ValueError("MODE '{}'' not recognized".format(mode))
    
    unit=find_key("UNIT",header).split()[0]
    
    if unit!="MM":
        raise ValueError("UNIT '{}' not recognized".format(unit))
        
    gcat=find_key("GCAT",header)
    

    # Separar las superficies en una lista de diccionarios
    surflist=[]
    for line in surfaces:
        #Ignore empty lines
        if line.split()==[]:
            continue
        if line.startswith("SURF"):
            surflist.append(dict())
            continue
        line=line.lstrip()
        code=line[:4]
        data=line[5:].split()
        data=[convert(d) for d in data]
        surflist[-1][code]=data

    #Eliminar el plano objeto y el plano imagen

    surflist.pop(0)
    surflist.pop()


    # Identificar el tipo de lentes a partir de el numero de superficies
    # validas

    ns=len(surflist)
    
    # Normal Singlets - Need to fix for asferical lenses
    if ns==2 and ("TYPE" not in surflist[0]) and ("TYPE" not in surflist[1]) and \
    key not in ["ACA254-060-B"]: # Lista de lentes que parecen tener un error
        c0=surflist[0]["CURV"][0]
        c1=surflist[1]["CURV"][0]
        d0=surflist[0]["DISZ"][0]
        #Not all lenses have DIAM in both surfaces
        if "DIAM" in surflist[0] and "DIAM" in surflist[1]:
            r0=surflist[0]["DIAM"][0]
            r1=surflist[1]["DIAM"][0]
        elif "DIAM" in surflist[0]:
            r0=surflist[0]["DIAM"][0]
            r1=r0
        elif "DIAM" in surflist[1]:
            r0=surflist[1]["DIAM"][0]
            r1=r0
        else:
            raise ValueError("'DIAM' not defined in the surfaces")
        
        g0=surflist[0]["GLAS"][0]
        #m0=get_material(g0)


        ##c|Verificar que las superficies son iguales, si no emitir un error
        assert r0==r1
        lens_data["material"] = g0
        lens_data["glass_catalogs"] = gcat #This is not used for the moment
        lens_data["thickness"] = surflist[0]["DISZ"][0]
        lens_data["radius"] = r0
        lens_data["curvature_s2"] = surflist[1]["CURV"][0]
        lens_data["curvature_s1"] = surflist[0]["CURV"][0]
        lens_data["type"] = "SphericalLens"
        
        pyot[key]=lens_data
        return pyot
        
        ##return CL.SphericalLens(r0,d0,c0,c1,material=m0)
    
     
    elif ns==3 and 'TYPE' not in surflist[0] and 'TYPE' not in surflist[1] and 'TYPE' not in surflist[2]: #Doublets, no asphericals
        lens_data["curvature_s1"]=surflist[0]["CURV"][0]
        lens_data["curvature_s2"]=surflist[1]["CURV"][0]
        lens_data["curvature_s3"]=surflist[2]["CURV"][0]
        lens_data["thickness_l1"]=surflist[0]["DISZ"][0]
        lens_data["thickness_l2"]=surflist[1]["DISZ"][0]
        r0=surflist[0]["DIAM"][0]
        r1=surflist[1]["DIAM"][0]
        r2=surflist[2]["DIAM"][0]
        
        #Verificar que las superficies son iguales, si no emitir un error
        assert r0==r1 and r1== r2
        
        lens_data["radius"] = r0
        
        lens_data["material_l1"]=surflist[0]["GLAS"][0]
        lens_data["material_l2"]=surflist[1]["GLAS"][0]
        lens_data["glass_catalogs"] = gcat 
        #return CL.Doublet(r0,c0,c1,c2, d0,d1,m0,m1)
        lens_data["type"] = "Doublet"
        pyot[key]=lens_data
        return pyot
    
    elif ns==4 and "GLAS" not in surflist[1] and \
         'TYPE' not in surflist[0] and 'TYPE' not in surflist[1] and \
         'TYPE' not in surflist[2] and 'TYPE' not in surflist[3]: #Dobletes con espaciado en Aire
        lens_data["curvature_s1"]=surflist[0]["CURV"][0]
        lens_data["curvature_s2"]=surflist[1]["CURV"][0]
        lens_data["curvature_s3"]=surflist[2]["CURV"][0]
        lens_data["curvature_s4"]=surflist[3]["CURV"][0]

        lens_data["thickness_l1"]=surflist[0]["DISZ"][0]
        lens_data["air_gap"]=surflist[1]["DISZ"][0]
        lens_data["thickness_l2"]=surflist[2]["DISZ"][0]

        #Verificar que las superficies son iguales, si no emitir un error
        #assert r0==r1 and r1== r2 and r2 ==r3 Esto no siempre se cumple

        r0=surflist[0]["DIAM"][0]
        r1=surflist[1]["DIAM"][0]
        r2=surflist[2]["DIAM"][0]
        r3=surflist[3]["DIAM"][0]
        
        lens_data["radius"] = r0

        lens_data["material_l1"]=surflist[0]["GLAS"][0]
        # g1=surflist[1]["GLAS"][0] Este parametro no existe. Es aire
        lens_data["material_l2"]=surflist[2]["GLAS"][0]

        lens_data["glass_catalogs"] = gcat 


        #return CL.AirSpacedDoublet(r0,c0,c1,c2,c3, d0,d1,d2,m0,m1)
        lens_data["type"] = "AirSpacedDoublet"
        pyot[key]=lens_data
        return pyot
    """
    else:
        for i in surflist:
            print("*", i)
        raise ValueError # Esto toca arreglarlo y generar un error que realmente indique que está pasando
    """


In [60]:
d=zmf2dict("THORLABS.ZMF")

import configparser

components=configparser.ConfigParser()

for k in d:
    z=zmx2pyoptools(d,k)
    if z is not None:
        components[k]=z[k]

with open('spherical.ini', 'w') as configfile:
    components.write(configfile)

In [55]:
z

{'LA7753-G': {'description': 'Ø1" ZnSe Plano-Convex Lens, f = 1000.0 mm, AR-Coated: 7-12µm',
  'material': 'ZNSE',
  'glass_catalogs': 'INFRARED',
  'thickness': 2.057495448839,
  'radius': 12.7,
  'curvature_s2': 0.0,
  'curvature_s1': 0.0007129303795968849}}

In [61]:
d

{'LK4836': 'VERS 150730\nMODE SEQ\nNAME LK4836 - Plano-Concave Cylindrical Lens - UV Fused Silica\nNOTE 0 FOR INFORMATION ONLY, NOT FOR MANUFACTURING.  ALL VALUES AND COATING(S) ARE TYPICAL AND THEORETICAL, ACTUAL PERFORMANCE MAY VARY.  FOR FURTHER INFORMATION CONTACT CUSTOMER SERVICE AT TECHSUPPORT@THORL\nNOTE 0 ABS.COM\nPFIL 0 0 0\nLANG 0\nUNIT MM X W X CM MR CPMM\nENPD 8.331225065605\nENVD 2.0E+1 1 0\nGFAC 0 0\nGCAT SCHOTT INFRARED MISC\nSDMA 0 1 0\nFTYP 0 0 1 3 0 0 0\nROPD 2\nPICB 1\nXFLN 0\nYFLN 0\nFWGN 1\nWAVM 1 4.861E-1 1\nWAVM 2 5.876E-1 1\nWAVM 3 6.563E-1 1\nPWAV 2\nGLRS 1 0\nSURF 0\n  FIMP \n\n  CURV 0.0\n  DISZ INFINITY\nSURF 1\n  COMM LK4836\n  STOP\n  TYPE TOROIDAL\n  FIMP \n\n  CURV -8.695652173913040500E-002\n  PARM 0 0\n  PARM 1 0\n  PARM 2 0\n  PARM 3 0\n  PARM 4 0\n  PARM 5 0\n  PARM 6 0\n  PARM 7 0\n  PARM 8 0\n  XDAT 1 0.000000000000E+000 0 0 1.000000000000E+000 0.000000000000E+000 0 ""\n  XDAT 2 1.000000000000E+002 0 0 1.000000000000E+000 0.000000000000E+000 0 ""\n